# Simulations to test GroupHug vs. 'one person, one vote' in case of maximum disagreement between voter groups.

The perform_comparison function below configures a GroupHug mechanism with a specified number of four different voter types - experts, intellectuals, participants and uneducated community members. In this simulation, each group is in favor of a different candidate. The experts prefer the expert candidate, the intellectuals prefer the intellectual candidate, ...and so on.

Furthermore, a SingleChoiceWeightedPlurality mechanism is set up with the same total number of voters - and uniform weights.

Each mechanism calculates a set of scores and announces a winner.

In [81]:
import sys
sys.path.append("..")

from mechanisms.group_hug_mechanism import GroupHug
from mechanisms.single_choice_weighted_plurality import SingleChoiceWeightedPlurality

def str_dict(d):
    return str(dict(sorted(d.items())))

def perform_comparison(gh, ecount, icount, pcount, ucount):

    expert_type = {"FELLOWSHIP_COMM": True}
    intellectual_type = {"FUND_MOD_1": True}
    participant_type = {"LIVE_TRACK_1": True}
    uneducated_type = {}

    voters = {}
    voters_choices = {}

    for i in range(ecount):
        voters["e" + str(i)] = expert_type
        voters_choices["e" + str(i)] = "Expert candidate"

    for i in range(icount):
        voters["i" + str(i)] = intellectual_type
        voters_choices["i" + str(i)] = "Intellectual candidate"

    for i in range(pcount):
        voters["p" + str(i)] = participant_type
        voters_choices["p" + str(i)] = "Participant candidate"
        
    for i in range(ucount):
        voters["u" + str(i)] = uneducated_type
        voters_choices["u" + str(i)] = "Uneducated candidate"

    uniform_voters = {id: {"weight": 1.0} for id in voters.keys()}

    print("VOTERS: " + str(ecount) + " experts, " + str(icount) + " intellectuals, " + str(pcount) + " participants, " + str(ucount) + " uneducated.")

    w_gh, r_gh = gh.calculate(voters, voters_choices)

    print("\nGroupHug Winner: " + str(w_gh))
    print("GroupHug Scores: " + str_dict(r_gh) + "\n")

    SCWP = SingleChoiceWeightedPlurality()
    w_scwp, r_scwp = SCWP.calculate(uniform_voters, voters_choices)

    print("Popularity contest winner: " + str(w_scwp))
    print("Popularity contest scores: " + str_dict(r_scwp))

Let's first use a default GroupHug mechanism (all voter groups have equal weight, default NFT weights are used.)

In [82]:
GH_def = GroupHug()

In our first example, below, only experts and uneducated community members participate (no intellectuals or participants.)

We see that a small expert group override a very large group of uneducated voters because the default GroupHug caps the influence of any of the N participating voter groups at 1/N, and the experts also influence the community vote:

In [83]:
perform_comparison(GH_def, 1, 0, 0, 500)

VOTERS: 1 experts, 0 intellectuals, 0 participants, 500 uneducated.

Experts: {"Experts' choice": 100.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0, 'Uneducated choice': 0}
Participants: {"Experts' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 0.2, 'Uneducated choice': 99.8}

GroupHug Winner: Experts' choice
GroupHug Scores: {"Experts' choice": 50.1, 'Uneducated choice': 49.9}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 1.0, 'Uneducated choice': 500.0}


If, at this point, a single 'intellectual' enters a vote (disagreeing with both experts and uneducated community members), the expert and the intellectual have equal weight. 

NB: WINNER UNDEFINED, 'RANDOM' BEHAVIOR!

In [84]:
perform_comparison(GH_def, 1, 1, 0, 100)

VOTERS: 1 experts, 1 intellectuals, 0 participants, 100 uneducated.

Experts: {"Experts' choice": 100.0, "Intellectuals' choice": 0.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0.0, "Intellectuals' choice": 100.0, 'Uneducated choice': 0.0}
Participants: {"Experts' choice": 0, "Intellectuals' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 1.0, "Intellectuals' choice": 1.0, 'Uneducated choice': 98.0}

GroupHug Winner: Experts' choice
GroupHug Scores: {"Experts' choice": 33.7, "Intellectuals' choice": 33.7, 'Uneducated choice': 32.7}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 1.0, "Intellectuals' choice": 1.0, 'Uneducated choice': 100.0}


The undefined behavior in case of equality of points is a problem that needs to be solved!

But there is another problem: In the default GroupHug mechanism, 1 expert has the same power as 1 student. We can change this by using custom group weights...

In [85]:
GH_custom_a = GroupHug(experts_group_weight = 30,
                     intellectuals_group_weight = 25,
                     participants_group_weight = 25,
                     community_group_weight = 20)


With the custom GroupHug mechanism defined above, 1 expert outweights 1 intellectual:

In [86]:
perform_comparison(GH_custom_a, 1, 1, 0, 100)

VOTERS: 1 experts, 1 intellectuals, 0 participants, 100 uneducated.

Experts: {"Experts' choice": 100.0, "Intellectuals' choice": 0.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0.0, "Intellectuals' choice": 100.0, 'Uneducated choice': 0.0}
Participants: {"Experts' choice": 0, "Intellectuals' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 1.0, "Intellectuals' choice": 1.0, 'Uneducated choice': 98.0}

GroupHug Winner: Experts' choice
GroupHug Scores: {"Experts' choice": 40.3, "Intellectuals' choice": 33.6, 'Uneducated choice': 26.1}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 1.0, "Intellectuals' choice": 1.0, 'Uneducated choice': 100.0}


In fact, 3 experts outweight 30 students and 100 uneducated:

In [87]:
perform_comparison(GH_custom_a, 3, 30, 0, 100)

VOTERS: 3 experts, 30 intellectuals, 0 participants, 100 uneducated.

Experts: {"Experts' choice": 100.0, "Intellectuals' choice": 0.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0.0, "Intellectuals' choice": 100.0, 'Uneducated choice': 0.0}
Participants: {"Experts' choice": 0, "Intellectuals' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 2.3, "Intellectuals' choice": 22.6, 'Uneducated choice': 75.2}

GroupHug Winner: Experts' choice
GroupHug Scores: {"Experts' choice": 40.6, "Intellectuals' choice": 39.3, 'Uneducated choice': 20.0}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 3.0, "Intellectuals' choice": 30.0, 'Uneducated choice': 100.0}


But there is still a limit to the experts' power. 

For example, 50 students (intellectuals) outweigh 3 experts:

In [88]:
perform_comparison(GH_custom_a, 3, 50, 0, 100)

VOTERS: 3 experts, 50 intellectuals, 0 participants, 100 uneducated.

Experts: {"Experts' choice": 100.0, "Intellectuals' choice": 0.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0.0, "Intellectuals' choice": 100.0, 'Uneducated choice': 0.0}
Participants: {"Experts' choice": 0, "Intellectuals' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 2.0, "Intellectuals' choice": 32.7, 'Uneducated choice': 65.4}

GroupHug Winner: Intellectuals' choice
GroupHug Scores: {"Experts' choice": 40.5, "Intellectuals' choice": 42.0, 'Uneducated choice': 17.4}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 3.0, "Intellectuals' choice": 50.0, 'Uneducated choice': 100.0}


Of course, we can move the threshold by changing the group weights slightly more in favor of the experts...

In [89]:
GH_custom_b = GroupHug(experts_group_weight = 32,
                     intellectuals_group_weight = 24,
                     participants_group_weight = 24,
                     community_group_weight = 20)

perform_comparison(GH_custom_b, 3, 50, 0, 100)

VOTERS: 3 experts, 50 intellectuals, 0 participants, 100 uneducated.

Experts: {"Experts' choice": 100.0, "Intellectuals' choice": 0.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0.0, "Intellectuals' choice": 100.0, 'Uneducated choice': 0.0}
Participants: {"Experts' choice": 0, "Intellectuals' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 2.0, "Intellectuals' choice": 32.7, 'Uneducated choice': 65.4}

GroupHug Winner: Experts' choice
GroupHug Scores: {"Experts' choice": 42.6, "Intellectuals' choice": 40.2, 'Uneducated choice': 17.2}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 3.0, "Intellectuals' choice": 50.0, 'Uneducated choice': 100.0}


Now, it takes even more student votes to override the experts' opinion:

In [90]:
perform_comparison(GH_custom_b, 3, 75, 0, 100)

VOTERS: 3 experts, 75 intellectuals, 0 participants, 100 uneducated.

Experts: {"Experts' choice": 100.0, "Intellectuals' choice": 0.0, 'Uneducated choice': 0.0}
Intellectuals: {"Experts' choice": 0.0, "Intellectuals' choice": 100.0, 'Uneducated choice': 0.0}
Participants: {"Experts' choice": 0, "Intellectuals' choice": 0, 'Uneducated choice': 0}
Community: {"Experts' choice": 1.7, "Intellectuals' choice": 42.1, 'Uneducated choice': 56.2}

GroupHug Winner: Intellectuals' choice
GroupHug Scores: {"Experts' choice": 42.6, "Intellectuals' choice": 42.7, 'Uneducated choice': 14.8}

Popularity contest winner: Uneducated choice
Popularity contest scores: {"Experts' choice": 3.0, "Intellectuals' choice": 75.0, 'Uneducated choice': 100.0}
